In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

In [3]:
import os
import sys
module_path = os.path.join(os.getcwd(), '..')
sys.path.append(module_path)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "deepset/roberta-large-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device=device, dtype=torch.bfloat16)

In [5]:
from lrp_engine import LRPEngine, checkpoint_hook
from lrp_engine.lrp_graph import make_graph

In [6]:
lrp = LRPEngine(topk=1, use_gamma=True, dtype=torch.bfloat16)

# SQuADv2 Benchmark

In [8]:
from datasets import load_dataset

dataset = load_dataset("squad_v2")

In [9]:
# QA version

from tqdm import tqdm
import json
import pickle

def run_roberta_squad_benchmark(model, tokenizer, lrp, dataset, run_name, question_first=True, model_topk=2):
    results = []
    top1_label_hits = 0
    top1_model_hits = 0
    total_examples = 0
    total_strict_intersect = 0
    total_span_intersect = 0
    total_strict_union = 0
    total_span_union = 0
    model_exact_matches = 0
    lrp_model_exact_matches = 0
    precision_denom = 0
    recall_denom = 0
    total_overlap = 0
    total_overlap_ratio = 0
    total_start_end_skips = 0
    total_empty_answer_skips = 0
    total_unanswerable = 0
    for example in tqdm(dataset["validation"]):
        question = example["question"]
        context = example["context"]
        answers = example["answers"]["text"]
        answer_start_inds = example["answers"]["answer_start"]
    
        if not answers:
            total_unanswerable += 1
            continue
    
        answer_char_ranges = [ (i, i + len(a)) for (a, i) in zip(answers, answer_start_inds) ]
    
        if question_first:
            tokenized = tokenizer(question, context, return_tensors="pt", return_offsets_mapping=True)
            input_ids = tokenized["input_ids"].to(device)
            offset_mapping = tokenized["offset_mapping"]
        else:
            tokenized = tokenizer(context, question, return_tensors="pt", return_offsets_mapping=True)
            input_ids = tokenized["input_ids"].to(device)
            offset_mapping = tokenized["offset_mapping"]
        if input_ids.shape[-1] > 512:
            continue
    
        output = model(input_ids.to(device))
    
        # Tokenizer outputs <s><question></s></s<context></s>
        # We need to mask the question tokens in the start/end logits, this is standard in QA evals
        sep_token_ind = list(input_ids[0]).index(tokenizer.eos_token_id)
        mask = torch.zeros_like(output.start_logits).bool()
        if question_first:
            mask_slice = [slice(None, None), slice(sep_token_ind + 1, None)]
        else:
            mask_slice = [slice(None, None), slice(None, sep_token_ind + 1)]
        mask[mask_slice] = True
    
        masked_start_logits = output.start_logits.masked_fill(~mask, -float("inf"))
        masked_end_logits = output.end_logits.masked_fill(~mask, -float("inf"))
        start = torch.argmax(masked_start_logits)
        end = torch.argmax(masked_end_logits)
    
        if start > end:
            total_start_end_skips += 1
            continue

        # Get the decoded answer per the model
        model_answer = tokenizer.decode(input_ids[0][start:end + 1], skip_special_tokens=False).replace("<unk>", "x").strip()
        if model_answer == "":
            total_empty_answer_skips += 1
            continue
    
        # Convert from token inds to char inds since that's how the dataset tracks the labels
        char_start = offset_mapping[0][start][0]
        if context[char_start] == " ":
            char_start += 1
        char_end = offset_mapping[0][end][1]

        # Take the answer where the prediction gets the best # chars intersecting over the model and label ranges divided by total # chars in label
        # best_overlap_ratio is the Jaccard similarity
        best_overlap, best_answer_len, best_overlap_ratio = max([ (ans_overlap := (max(0, min(char_end, answer_end) - max(char_start, answer_start))), ans_len := (answer_end - answer_start), ans_overlap / (ans_len + len(model_answer) - ans_overlap)) for (answer_start, answer_end) in answer_char_ranges ], key =lambda x: x[2])
        total_overlap += best_overlap
        total_overlap_ratio += best_overlap_ratio
        precision_denom += len(model_answer)
        recall_denom += best_answer_len
    
        if model_exact_match := (best_overlap_ratio == 1):
            model_exact_matches += 1

        # Now run LRP
        lrp_input1, lrp_input2 = output.start_logits, output.end_logits
        lrp_input1[0][0] = 0
        lrp_input2[0][0] = 0
        checkpoint_vals, param_vals = lrp.run((lrp_input1, lrp_input2))

        lrp_max = param_vals[2].masked_fill(~mask, -float("inf")).flatten().argmax()
        lrp_top_token = tokenizer.decode([input_ids[0][lrp_max]], skip_special_tokens=True).strip().replace(chr(9601), "")

        # Do model answer-based accuracy, i.e. is the attribution aligned with the model prediction
        if start <= lrp_max <= end:
            top1_model_hits += 1

        # Do IoU with the model prediction
        # We split into two ways to compute:
        # For Strict IoU we do not count tokens IN BETWEEN start and end because the model is meant to only output 2 signals for start and end.
        # So, strict_intersect = 2, strict_union = 2 if LRP top-2 and the model prediction indices matched exactly
        # strict_intersect = 1, strict_union = 3 if LRP only 1 of the LRP top-2 was a model prediction index
        # strict_intersect = 0, strict_union = 4 if neither LRP top-2 were a model prediction index
        # We double count in the case of single word answers (start = end), and do not consider the second place attributed token if the first
        #   place token was the answer.
        # For Span IoU, we DO count tokens in between start and end
        strict_intersect = 0
        span_intersect = 0
        strict_union = model_topk
        span_union = model_topk
        found_start = False
        found_end = False
        for top_ind in param_vals[2].flatten().topk(model_topk).indices:
            if found_start and found_end:
                break
            if start == top_ind:
                found_start = True
                strict_intersect += 1
                span_intersect += 1

            if end == top_ind:   # Crucial to keep these two as separate ifs for single-word answers
                found_end = True
                strict_intersect += 1
                span_intersect += 1

            if start < top_ind < end:
                strict_union += 1
                span_intersect += 1

            elif top_ind < start or end < top_ind:
                strict_union += 1
                span_union += 1

        if lrp_model_exact_match := (strict_intersect == strict_union):
            lrp_model_exact_matches += 1
        total_strict_intersect += strict_intersect
        total_strict_union += strict_union
        total_span_intersect += span_intersect
        total_span_union += span_union

        # Do label-based accuracy
        if any(lrp_top_token in ans for ans in answers):
            # Is the attribution aligned with the ground truth label
            top1_label_hits += 1

        lrp_top5 = param_vals[2].flatten().topk(k=5)
        lrp_top5_tokens = tokenizer.decode(input_ids[0][lrp_top5.indices.cpu()])
        results.append({
            "example": example,
            "model_exact_match": bool(model_exact_match),
            "lrp_top1_ind": lrp_max.tolist(),
            "model_start_end": (start.detach().cpu().tolist(), end.detach().cpu().tolist()),
            "lrp_top1_token": lrp_top_token,
            "lrp_top5_tokens": lrp_top5_tokens,
            "lrp_top5_relevances": lrp_top5.values.detach().cpu().tolist(),
            "lrp_model_strict_intersect": strict_intersect,
            "lrp_model_strict_union": strict_union,
            "lrp_model_span_intersect": span_intersect,
            "lrp_model_span_union": span_union,
            "lrp_model_exact_match": lrp_model_exact_match,
        })

        total_examples += 1
        if not (total_examples % 100):
            precision = float(total_overlap / precision_denom)
            recall = float(total_overlap / recall_denom)
            f1 = float(2 / ((1 / precision) + (1 / recall)))
            print("Total examples: ", total_examples)
            print("Model exact matches: ", model_exact_matches)
            print("Model exact match %: ", model_exact_matches / total_examples)
            print("Model precision: ", precision)
            print("Model recall: ", recall)
            print("Model F1: ", f1)
            print("Model overlap ratio: ", float(total_overlap_ratio / total_examples))
            print("LRP Top-1 model answer hits: ", top1_model_hits)
            print("LRP Top-1 model answer hit %: ", top1_model_hits / total_examples)
            print("LRP Top-1 label answer hits: ", top1_label_hits)
            print("LRP Top-1 label answer hit %: ", top1_label_hits / total_examples)
            print(f"LRP Top-{model_topk} Strict Accuracy: ", total_strict_intersect / (model_topk * total_examples))
            print(f"LRP Top-{model_topk} Span Accuracy: ", total_span_intersect / (model_topk * total_examples))
            print("LRP-Model exact matches: ", lrp_model_exact_matches)
            print("LRP-Model Strict IoU: ", total_strict_intersect / total_strict_union)
            print("LRP-Model Span IoU: ", total_span_intersect / total_span_union)
            print("Total start-end skips: ", total_start_end_skips)
            print("Total empty model answer skips: ", total_empty_answer_skips)
            print("Total unanswerable: ", total_unanswerable)

    precision = float(total_overlap / precision_denom)
    recall = float(total_overlap / recall_denom)
    f1 = float(2 / ((1 / precision) + (1 / recall)))
    data = {
        "summary": {
            "total_examples": total_examples,
            "model_exact_matches": model_exact_matches,
            "model_exact_match_pct": model_exact_matches / total_examples,
            "model_precision": precision,
            "model_recall": recall,
            "model_f1": f1,
            "model_overlap_ratio": float(total_overlap_ratio / total_examples),
            "lrp_top1_model_answer_hits": top1_model_hits,
            "lrp_top1_model_answer_hit_pct": top1_model_hits / total_examples,
            "lrp_top1_label_answer_hits": top1_label_hits,
            "lrp_top1_label_answer_hit_pct": top1_label_hits / total_examples,
            f"lrp_top{model_topk}_strict_accuracy": total_strict_intersect / (model_topk * total_examples),
            f"lrp_top{model_topk}_span_accuracy": total_span_intersect / (model_topk * total_examples),
            "lrp_model_exact_matches": lrp_model_exact_matches,
            "lrp_model_strict_iou": total_strict_intersect / total_strict_union,
            "lrp_model_span_iou": total_span_intersect / total_span_union,
            "total_start_end_skips" : total_start_end_skips,
            "total_empty_answer_skips" : total_empty_answer_skips,
            "total_unanswerable" : total_unanswerable,
        },
        "results": results
    }

    try:
        with open(f"roberta_squadv2_results_{run_name}.json", "w") as f:
            json.dump(data, f, indent=4)
    except TypeError:
        print("Failed JSON write, attempting pickle...")
        try:
            with open(f"roberta_squadv2_results_{run_name}.pkl", "wb") as f:
                pickle.dump(data, f)
        except Exception as e:
            print("Encountered exception while pickling, will return (data, exception)...")
            return data, e
    return data

In [33]:
results = run_roberta_squad_benchmark(model, tokenizer, lrp, dataset, "gamma")

  2%|█▍                                                                            | 218/11873 [00:29<24:56,  7.79it/s]

Total examples:  100
Model exact matches:  83
Model exact match %:  0.83
Model precision:  0.8022130131721497
Model recall:  0.9243028163909912
Model F1:  0.858941151565508
Model overlap ratio:  0.888780951499939
LRP Top-1 model answer hits:  96
LRP Top-1 model answer hit %:  0.96
LRP Top-1 label answer hits:  94
LRP Top-1 label answer hit %:  0.94
LRP Top-2 Strict Accuracy:  0.89
LRP Top-2 Span Accuracy:  0.945
LRP-Model exact matches:  82
LRP-Model Strict IoU:  0.8018018018018018
LRP-Model Span IoU:  0.8957345971563981
Total start-end skips:  1
Total empty model answer skips:  0
Total unanswerable:  117


  4%|██▊                                                                           | 434/11873 [00:59<18:59, 10.04it/s]

Total examples:  200
Model exact matches:  155
Model exact match %:  0.775
Model precision:  0.7599042654037476
Model recall:  0.8984596133232117
Model F1:  0.8233938295634095
Model overlap ratio:  0.8675488829612732
LRP Top-1 model answer hits:  189
LRP Top-1 model answer hit %:  0.945
LRP Top-1 label answer hits:  182
LRP Top-1 label answer hit %:  0.91
LRP Top-2 Strict Accuracy:  0.885
LRP Top-2 Span Accuracy:  0.93
LRP-Model exact matches:  165
LRP-Model Strict IoU:  0.7937219730941704
LRP-Model Span IoU:  0.8691588785046729
Total start-end skips:  3
Total empty model answer skips:  0
Total unanswerable:  231


  5%|████▏                                                                         | 639/11873 [01:28<41:24,  4.52it/s]

...

 98%|██████████████████████████████████████████████████████████████████████████▏ | 11594/11873 [27:32<00:51,  5.46it/s]

Total examples:  5700
Model exact matches:  4688
Model exact match %:  0.8224561403508772
Model precision:  0.8150673508644104
Model recall:  0.9143727421760559
Model F1:  0.8618689617144503
Model overlap ratio:  0.9052829742431641
LRP Top-1 model answer hits:  5564
LRP Top-1 model answer hit %:  0.976140350877193
LRP Top-1 label answer hits:  5349
LRP Top-1 label answer hit %:  0.9384210526315789
LRP Top-2 Strict Accuracy:  0.8907017543859649
LRP Top-2 Span Accuracy:  0.9639473684210527
LRP-Model exact matches:  4597
LRP-Model Strict IoU:  0.8029416416258105
LRP-Model Span IoU:  0.9304038608077216
Total start-end skips:  41
Total empty model answer skips:  0
Total unanswerable:  5814


 99%|███████████████████████████████████████████████████████████████████████████▍| 11790/11873 [28:01<00:09,  8.98it/s]

Total examples:  5800
Model exact matches:  4774
Model exact match %:  0.823103448275862
Model precision:  0.816024661064148
Model recall:  0.9146488904953003
Model F1:  0.8625266737179793
Model overlap ratio:  0.9053937196731567
LRP Top-1 model answer hits:  5660
LRP Top-1 model answer hit %:  0.9758620689655172
LRP Top-1 label answer hits:  5443
LRP Top-1 label answer hit %:  0.9384482758620689
LRP Top-2 Strict Accuracy:  0.890603448275862
LRP Top-2 Span Accuracy:  0.9635344827586206
LRP-Model exact matches:  4678
LRP-Model Strict IoU:  0.8027818789338721
LRP-Model Span IoU:  0.9296348665058638
Total start-end skips:  45
Total empty model answer skips:  0
Total unanswerable:  5906


100%|████████████████████████████████████████████████████████████████████████████| 11873/11873 [28:13<00:00,  7.01it/s]


# Dummy Test (Don't run if you want to run the full SQuADv2 benchmark)

In [8]:
context = "Welcome to the final examination for this term's offering of CS100. Please remove all headphones and earbuds, as well as hats and hoods. Place your bag under your desk so that it does not block the aisle. You are permitted writing instruments, a clear water bottle, and any aids listed on the front of your booklet. The exam will be 150 minutes in duration. You may now begin."
question = "What is this?"

In [9]:
input_ids = tokenizer(question, context, return_tensors="pt")["input_ids"]

In [10]:
output = model(input_ids.to(device))

In [11]:
g = make_graph(output.start_logits)
g[2], len(g[2]), g[4]

({'CloneBackward0': 1,
  'SqueezeBackward1': 1,
  'SplitBackward0': 1,
  'ViewBackward0': 386,
  'AddBackward0': 195,
  'AccumulateGrad': 391,
  'MmBackward0': 145,
  'NativeLayerNormBackward0': 49,
  'GeluBackward0': 24,
  'TransposeBackward0': 24,
  'ScaledDotProductEfficientAttentionBackward0': 24,
  'PermuteBackward0': 72,
  'EmbeddingBackward0': 3,
  'TBackward0': 145},
 14,
 1461)

In [10]:
start = torch.argmax(output.start_logits)
end = torch.argmax(output.end_logits) + 1

answer_tokens = input_ids[0][start:end]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
print(answer)

 final examination


In [10]:
checkpoint_vals, param_vals = lrp.run((output.start_logits, output.end_logits))

In [11]:
checkpoint_vals1, param_vals1 = lrp.run((output.start_logits, output.end_logits))

In [12]:
# Check results diff between 1st pass and 2nd pass on same input (should be ~0)
[
    ((p1 - p0)**2).sum() for p1, p0 in zip(param_vals1, param_vals)
]

[tensor(0., device='cuda:0'),
 tensor(0., device='cuda:0'),
 tensor(0., device='cuda:0')]

In [13]:
# Top 5 tokens from LRP
lrp_answer_ids = input_ids[0][param_vals[1].flatten().topk(k=5).indices.cpu()]
print(tokenizer.decode(lrp_answer_ids))

Welcome hooduds aisle booklet


In [ ]:
from tqdm import tqdm

results = []

for example in tqdm(dataset["validation"].select(range(100))):
    question = example["question"]
    context = example["context"]
    answers = example["answers"]["text"]

    input_ids = tokenizer(question, context, return_tensors="pt")["input_ids"]
    output = model(input_ids.to(device))

    start = torch.argmax(output.start_logits)
    end = torch.argmax(output.end_logits) + 1
    model_answer = tokenizer.decode(input_ids[0][start:end], skip_special_tokens=True)

    checkpoint_vals, param_vals = lrp.run((output.start_logits, output.end_logits))

    lrp_top5 = param_vals[1].flatten().topk(k=5)
    lrp_top5_tokens = tokenizer.decode(input_ids[0][lrp_top5.indices.cpu()], skip_special_tokens=True)
    lrp_start_end = param_vals[1].flatten().topk(k=2).indices.cpu().sort()
    lrp_start = lrp_start_end[0][0]
    if lrp_start == 0:
        lrp_start = lrp_start_end[0][1]
    lrp_end = lrp_start_end[0][-1]
    lrp_answer_ids = input_ids[0][lrp_start:lrp_end + 1]
    lrp_answer = tokenizer.decode(lrp_answer_ids, skip_special_tokens=True)

    results.append({
        "example": example,
        "model_answer": model_answer,
        "lrp_answer": lrp_answer,
        "lrp_top5_tokens": lrp_top5_tokens,
        "lrp_top5_relevances": lrp_top5.values.cpu(),
        "is_impossible": len(answers) == 0
    })

In [ ]:
for res in results:
    print("Q: ", res["example"]["question"])
    print("A (labels): ", res["example"]["answers"]["text"])
    print("Model: ", res["model_answer"])
    print("LRP: ", res["lrp_answer"])
    print("LRP top5: ", res["lrp_top5_tokens"])
    print("LRP top5 attributions: ", res["lrp_top5_relevances"], '\n')